# Obtaining the gradient and training (optimizing) the parameters

In this tutorial, we will describe how you can use JAX's automatic differentiation to obtain gradients through `neurax` simulations and how you can use optimize the parameters with the Adam optimizer.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# I have experienced stability issues with float32.
from jax.config import config
config.update("jax_enable_x64", True)
config.update("jax_platform_name", "cpu")

import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]=".8"

In [3]:
import time
import matplotlib.pyplot as plt
import numpy as np
import jax
import jax.numpy as jnp
from jax import jit, value_and_grad

import neurax as nx
from neurax.channels import HHChannel
from neurax.synapses import GlutamateSynapse

### Setup

In [4]:
# Number of segments per branch.
nseg_per_branch = 8

# Stimulus.
i_delay = 3.0  # ms
i_amp = 0.05  # nA
i_dur = 2.0  # ms

# Duration and step size.
dt = 0.025  # ms
t_max = 50.0  # ms

In [5]:
time_vec = jnp.arange(0.0, t_max+dt, dt)

### Define stimuli and recordings

In [6]:
recs = [nx.Recording(cell_ind, 1, 0.0) for cell_ind in range(5)]
stims = [
    nx.Stimulus(stim_ind, 1, 0.0, current=nx.step_current(i_delay, i_dur, i_amp, time_vec)) for stim_ind in range(2)
]

### Let's define a network

In [7]:
comp = nx.Compartment()
branch = nx.Branch([comp for _ in range(nseg_per_branch)])
cell = nx.Cell([branch for _ in range(5)], parents=jnp.asarray([-1, 0, 0, 1, 1]))

In [8]:
_ = np.random.seed(0)
conn_builder = nx.ConnectivityBuilder([cell.total_nbranches for _ in range(5)])
connectivities = [nx.Connectivity(GlutamateSynapse(), conn_builder.fc(np.arange(0, 2), np.arange(2, 5)))]

In [9]:
network = nx.Network([cell for _ in range(5)], connectivities)

In [10]:
network.insert(HHChannel())

### Defining trainable parameters

This follows the same API as `.set_params()` seen in the previous tutorial. If you want to use a single parameter for all `radius`es in the entire network, do:

In [11]:
network.make_trainable("radius")

We can also define parameters for individual compartments. To do this, use the `"all"` key. The following defines a separate parameter the sodium conductance for every compartment in the entire network:

In [12]:
network.cell("all").branch("all").comp("all").make_trainable("gNa")

### Making synaptic parameters trainable

Synaptic parameters can be made trainable in the exact same way. To use a single parameter for all syanptic conductances in the entire network, do

In [13]:
network.make_trainable("gS")

and to use a different syanptic conductance for all syanpses, do

In [14]:
network.GlutamateSynapse("all").make_trainable("gS")

### Running the simulation again

Once all parameters are defined, you have to use `.get_parameters()` to obtain all trainable parameters. This is also the time to check how many trainable parameters your network has:

In [15]:
params = network.get_parameters()
# print(params)

You can now run the simulation with the trainable parameters by passing them to the `nx.integrate` function.

In [16]:
s = nx.integrate(network, stimuli=stims, recordings=recs, delta_t=dt, params=params)

### Defining a loss function

Let us define a loss function to be optimized:

In [36]:
def loss(params):
    params = transform.forward(params)
    s = nx.integrate(network, stimuli=stims, recordings=recs, delta_t=dt, params=params)
    return jnp.sum(s[0, -1])

And we can use `JAX`'s inbuilt functions to take the gradient through the entire ODE:

In [37]:
jitted_grad = jit(value_and_grad(loss))

In [38]:
value, gradient = jitted_grad(params)

### Training

We will use the ADAM optimizer from the [optax library](https://optax.readthedocs.io/en/latest/) to optimize the free parameters (you have to install the package with `pip install optax` first):

In [39]:
import optax

Before training, however, we will enforce for all parameters to be within a prespecified range (such that, e.g., conductances can not become negative)

In [40]:
transform = nx.ParamTransform(
    lowers={"gNa": 0.05, "gK": 0.01, "gLeak": 0.0001, "radius": 0.1, "length": 1.0, "axial_resistivity": 500.0, "gS": 0.01}, 
    uppers={"gNa": 1.1, "gK": 0.3, "gLeak": 0.001, "radius": 5.0, "length": 20.0, "axial_resistivity": 5500.0, "gS": 5.0}, 
)

Let's modify the loss function acocrdingly:

In [41]:
def loss(params):
    params = transform.forward(params)
    s = nx.integrate(network, stimuli=stims, recordings=recs, delta_t=dt, params=params)
    return jnp.sum(s[0, -1])

jitted_grad = jit(value_and_grad(loss))
value, gradient = jitted_grad(params)

Then we define the optimizer:

In [42]:
opt_params = transform.inverse(params)
optimizer = optax.adam(learning_rate=1e-2)
opt_state = optimizer.init(opt_params)

In [43]:
epoch_losses = []

for epoch in range(5):
    loss_val, gradient = jitted_grad(opt_params)
    updates, opt_state = optimizer.update(gradient, opt_state)
    opt_params = optax.apply_updates(opt_params, updates)

    print(f"epoch {epoch}, loss {loss_val}")
    epoch_losses.append(loss_val)
    
final_params = transform.forward(opt_params)

epoch 0, loss -64.97740510297487
epoch 1, loss -64.98296369502924
epoch 2, loss -64.98846441030534
epoch 3, loss -64.99390672049375
epoch 4, loss -64.99929015505666


Indeed, the loss goes down.